# Selecting many objects based on physical parameters

The "First 100 numbered asteroids" example above in tutorial `3.3.2 Data Access` is rather artificial. Instead,
we might want to select asteroids based on shared physical properties. Using
the `Rock` class or the `rocks.rocks` function is not suitable here as we don't
know the parameters of the objects before we query them.

Instead, besides the single-object summary that is the <a href="https://ssp.imcce.fr/webservices/ssodnet/api/ssocard/">ssoCard</a>, SsODNet offers
the <a href="https://ssp.imcce.fr/webservices/ssodnet/api/ssobft/">*Big Flat Table* (BFT)</a> to access the attributes of *all* known asteroids.
This makes it easy to query for asteroids based on shared properties. In the example below,
we load this table and select all D-type asteroids.

In [1]:
import os

import rocks
import matplotlib.pyplot as plt

In [ ]:
# Load BFT as pandas DataFrame
bft = rocks.load_bft()

# Get asteroids that have been classified as D-type
dtypes = bft[bft['taxonomy.class'].isin(['D'])]
print(f"Found {len(dtypes)} asteroids.") 

Yes, you're right: We are using a `pandas` DataFrame here, and above we talked about the downsides of this.
However, we are currently not focused on a single object, but on a large number of them. It is perfectly fine here to think about
indexing a large table because that is in fact what we are doing: selecting based on shared properties.

Philosophical reasoning aside, the switch to the `pandas` DataFrame unfortunately comes with some tedious syntax again.
For example, to plot the proper semi-major axis versus the albedo of all our D-types, we write the following code.

In [ ]:
fig, ax = plt.subplots()

ax.scatter(dtypes["proper_elements.proper_semi_major_axis.value"], dtypes["albedo.value"], s=10)
ax.set(xlabel='Proper Semi-major Axis / au', ylabel='Albedo')

It's not pretty but it works well. And we can always switch back to using `Rock` representations of our objects (though I wouldn't recommend this
if you have more than ~500 objects):

In [ ]:
dtypes = dtypes[:20]  # only use 20 of them for execution time, it's a proof-of-concept
dtypes = rocks.rocks(dtypes['sso_name'])
dtypes

# Extending Catalogue Data

In this tutorial, we take a catalogue of asteroid data and extend it using the `astroquery` and `rocks` packages.
This is a common task when working with archival data and we will see frequently-occurring issues and how to solve them.

The catalogue we will be working with is the first version of the *Moving Objects Catalog* (MOC1) of the SDSS survey.

In [ ]:
import time

from astroquery.mpc import MPC
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rocks
from tqdm import tqdm  # for progress bars

import wget

First task: Acquire the catalogue. We can download it and read it from file (do
this if you expect to use it frequently), but for the purpose of demonstration,
we use a nice feature of the `pandas` package: Most data-ingestion functions
like `read_csv`, `read_json`, `read_fwf` (fixed-width format) accept URLs
pointing to files. Here, we point to a `gzip` compressed version of the
catalogue and provide `pandas` with some metadata to parse the data it is
downloading. We don't need all the information, we only get the number,
designation, and the ugriz magnitudes from the catalogue.

In [6]:
# Download the ADR1 data file if it doesn't exist
if not os.path.isfile("ADR1.dat.gz"):
    wget.download("https://faculty.washington.edu/ivezic/sdssmoc/ADR1.dat.gz", "ADR1.dat.gz")

In [ ]:
data = pd.read_fwf(
    "ADR1.dat.gz",
    colspecs=[(244, 250), (250, 270), (163, 168), (174, 179), (185, 190), (196, 201), (207, 212)],
    names=["numeration", "designation", 'u', 'g', 'r', 'i', 'z'],
    compression='gzip'
)

print(f"Number of observations in SDSS MOC1: {len(data)}")
data.head()

We note that there are asteroids with `numeration = 0` and `designation = '-'`. The [documentation](http://faculty.washington.edu/ivezic/sdssmoc/sdssmoc1.html) of this catalogue
states that only 12,602 of the 58,117 entries could be associated to 10,592 unique objects known at the time. The rest are included as unidentified objects. We could likely identify
the vast majority now using a tool like [SkyBoT](https://ssp.imcce.fr/webservices/skybot/) to check the SDSS field-of-views for asteroids known today, but this is not the purpose of this tutorial (and it has
been done already).

Instead, we drop all entries belonging to unknown asteroids here. We further set the number of unnumbered asteroids to `NaN` instead
of the commonly used (but aesthetically questionable) `0`.

In [ ]:
# Remove the unknown objects
data = data[data.designation.str.strip(" ") != "-"]
print(f"Observations of known objects: {len(set(data.designation))}")

# Unnumbered objects should be NaN
data.loc[data.numeration == 0, "numeration"] = np.nan

data.describe()

Looking good so far. Since the catalogue came out over 20 years ago, we can expect a large fraction of theses asteroids to now have been numbered
and even named. We thus add two columns, `name` and `number`. We query the current name and number using the `rocks.identify` function,
which accepts a list of identifiers (like designations, numbers, names) and returns a list of tuples containing the name and number of the identified
objects. We create the list of identifiers by combining the `numberation` and the `designation` columns.

In [ ]:
# Create list of identifiers by merging 'numeration' and 'designation' columns
ids = data.numeration.fillna(data.designation)
print("Identifying known objects in catalogue..")
names_numbers = rocks.identify(ids)

The names and numbers are returned in the order of the passed identifiers. We can add them to the SDSS data using a simple list comprehension.

In [ ]:
# Add numbers and names to data
data["name"] = [name_number[0] for name_number in names_numbers]
data["number"] = [name_number[1] for name_number in names_numbers]

# Print part of the result
data.number = data.number.astype("Int64")  # Int64 supports integers and NaN
data.head()

Great, this worked as expected.
We set the datatype of the `number` columns to the `Int64` type as this type can deal with integers and `NaN` values.
This is a useful little trick when working with the `number` column.

Now that we know what asteroids we are working with, let's have a look at their colours.

In [ ]:
fig, ax = plt.subplots()

ax.scatter(data["g"] - data["r"], data["i"] - data["z"], alpha=0.4, s=10, lw=0)
ax.set(xlabel="g-r", ylabel="i-z");

We have some outliers that we remove generously.

In [12]:
data = data[(data['g']-data['r']) < 2]
data = data[(data['g']-data['r']) > -2]
data = data[(data['i']-data['z']) < 2]
data = data[(data['i']-data['z']) > -2]

In [ ]:
fig, ax = plt.subplots()

ax.scatter(data["g"] - data["r"], data["i"] - data["z"], alpha=0.4, s=10, lw=0)
ax.set(xlabel="g-r", ylabel="i-z");

Now it looks as we expected it. Next, we add some additional information to this plot.
For each object, we query the albedo and the taxonomical complex from SsODNet using `rocks` and
the $\Delta$V from the MPC. `rocks` uses asynchronous queries, meaning that it will process multiple
queries at the same time, while the `astroquery` code uses synchronous code, meaning that the queries are processed
one after the other. We will see this difference in the execution time.

For the `MPC` query, we have to specify whether we query the object by name,
number, or designation. To simplify, we reduce the dataset to numbered
asteroids and only take 100 entries in the catalogue for the sake of brevity.
And for the sake of a fair comparison, we turn off the cache usage of `rocks`,
which would typically prevent repeated queries to the server in favour of local
copies of the data.

In [ ]:
data = data[~pd.isna(data.number)]
demo = data[:100]
rocks.CACHELESS = True

start_time = time.time()
targets = rocks.rocks(demo['name'])

for target in targets:
  demo.loc[demo['name'] == target.name, 'complex'] = target.taxonomy.complex.value
  demo.loc[demo['name'] == target.name, 'albedo'] = target.albedo.value
print(f"rocks took {time.time() - start_time:.2f} seconds.")

For the `astroquery.MPC` module, we cannot disable caching, so only the first run of this block
will give a representative result.

In [ ]:
start_time = time.time()
for ind, d in demo.iterrows():
  target = MPC.query_object('asteroid', number=d['number'])
  demo.loc[ind, 'delta_v'] = target[0]['delta_v']
print(f"astroquery took {time.time() - start_time:.2f} seconds.")

`rocks` is about 5-6 times faster due to the asynchronous query logic.

Since we removed a lot of data points to assess the different execution times, the plots
are less impressive than before.

In [ ]:
fig, ax = plt.subplots()

scat = ax.scatter(demo["g"] - demo["r"], demo["i"] - demo["z"], c=demo['albedo'], cmap='cool')
ax.set(xlabel="g-r", ylabel="i-z")
fig.colorbar(scat, label='Albedo');

In practice, there is no issue working with tens/hundreds of thousands of data points like this, as both
packages apply caching which significantly reduces the execution times on repeated runs.
<br>
If you are still exploring data and do not need full information sources, a cross-match with the 
<a href="https://ssp.imcce.fr/webservices/ssodnet/api/ssobft/">BFT</a>. Easy, fast, efficient!

In [17]:
demo2 = data.merge( bft, left_on="name", right_on="sso_name", how="left")

In [ ]:
fig, ax = plt.subplots()

scat = ax.scatter(demo2["g"] - demo2["r"], demo2["i"] - demo2["z"], c=demo2['albedo.value'], cmap='cool')
ax.set(xlabel="g-r", ylabel="i-z")
fig.colorbar(scat, label='Albedo');

Voilà!